Baseline Happiness:
mean  = 6.828
stdev = 1.761

Lonely Happiness:
mean  = 5.290
stdev = 2.348

Sick Happiness:
mean  = 3.204
stdev = 1.441

Healthy contacts/day:
mean  = 25.880
stdev = 19.396

Sick contacts/day:
mean  = 15.547
stdev = 12.598

Step 1: Build SIR Model for Olin
    - Use Survey data:
        - Contact Rate
        - Recovery Speed
        - Infection Rate
    
    
Step 2: Add Quarantine
    - Keep a working version without Quarrantine
    - I -> Q
    - Can only infect roommate?
        - 1 interaction per day


Step 3: Create Happines
    - Assign baseline happiness
    - Calculate lonliness and sadness modifiers
    - Stretch Goal: Stress Strain model to account for missing work/length of quarrantine
    

Step 4: Integrate Happiness
    - Being sick = less happy
    - Being alone = less happy
    - Being sick and alone = lonliness modifier * sickness modifier
    
    
Step 5: Add Quarrantine Agressiveness Params
    - Quarantine Lengths
    - False Positives for Symptoms?
    - Random Pre-Emptive Isolation???


Step 6: Sweep Quarantine Agressiveness
    - Multiple Line plots ranging from no quarantine to putting everyone in little platic bubbles from day 1
    - 3D plot for 2x sweep of length and preemptiveness 
    
    
Step 7: Everything is a normal distribution


Step 8: git push origin master

**Chapter 13 Slightly Derivative Stuff**

In [1]:
def make_system(beta, gamma):
    """Make a system object for the SIR model.
    
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    init = State(S=89, I=1, R=0)
    init /= np.sum(init)

    t0 = 0
    t_end = 7 * 14

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma)

In [2]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.row[t0] = init
    
    for t in linrange(t0, t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)
    
    return frame

In [3]:
def update_func(state, t, system):
    """Update the SIR model.
    
    state: State (s, i, r)
    t: time
    system: System object
    
    returns: State (sir)
    """
    s, i, r = state

    infected = system.beta * i * s    
    recovered = system.gamma * i
    
    s -= infected
    i += infected - recovered
    r += recovered
    
    return State(S=s, I=i, R=r)

In [4]:
def slope_func(state, t, system):
    """Update the SIR model.
    
    state: State (s, i, r)
    t: time
    system: System object
    
    returns: State (sir)
    """
    s, i, r = state

    dIdt = system.beta * i * s
    dSdt = -dIdt
    dRdt = system.gamma * i
    dIdt -= dRdt
    
    return dSdt, dIdt, dRdt

**Things That Are Fun, New, and Exciting!**

**Step 1**

In [ ]:
#code

**Step 2**

In [ ]:
#code

**Step 3**

In [ ]:
#code

**Step 4**

In [ ]:
#code

**Step 5**

In [ ]:
#code

**Step 6**

In [ ]:
#code

**Step 7**

In [ ]:
#code